In [1]:
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg'

In [2]:
%matplotlib notebook

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
from matplotlib.animation import FuncAnimation
import heapq

class DifferentialDriveRobot:
    def __init__(self, x, y, theta, wheel_base=0.3):
        self.x = x
        self.y = y
        self.theta = theta
        self.wheel_base = wheel_base
        self.path_history = [(x, y)]
        
    def update(self, v_left, v_right, dt):
        v = (v_right + v_left) / 2
        omega = (v_right - v_left) / self.wheel_base
        
        if abs(omega) < 1e-6:
            self.x += v * np.cos(self.theta) * dt
            self.y += v * np.sin(self.theta) * dt
        else:
            R = v / omega
            self.x += R * (np.sin(self.theta + omega * dt) - np.sin(self.theta))
            self.y += -R * (np.cos(self.theta + omega * dt) - np.cos(self.theta))
            self.theta += omega * dt
        
        self.theta = np.arctan2(np.sin(self.theta), np.cos(self.theta))
        self.path_history.append((self.x, self.y))

class Maze:
    def __init__(self, width=10, height=10):
        self.width = width
        self.height = height
        self.obstacles = []
        self.create_maze()
        
    def create_maze(self):
        self.obstacles.append(Rectangle((0, 0), self.width, 0.2))
        self.obstacles.append(Rectangle((0, self.height-0.2), self.width, 0.2))
        self.obstacles.append(Rectangle((0, 0), 0.2, self.height))
        self.obstacles.append(Rectangle((self.width-0.2, 0), 0.2, self.height))
        
        self.obstacles.append(Rectangle((2, 2), 3, 0.3))
        self.obstacles.append(Rectangle((6, 4), 0.3, 3))
        self.obstacles.append(Rectangle((3, 6), 2, 0.3))
        self.obstacles.append(Rectangle((7, 1), 2, 0.3))
        self.obstacles.append(Rectangle((4, 8), 0.3, 1.5))
        
    def is_collision(self, x, y, robot_radius=0.2):
        for obs in self.obstacles:
            ox, oy = obs.get_xy()
            ow, oh = obs.get_width(), obs.get_height()
            if (ox - robot_radius <= x <= ox + ow + robot_radius and
                oy - robot_radius <= y <= oy + oh + robot_radius):
                return True
        return False
    
    def create_grid(self, resolution=0.3):
        rows = int(self.height / resolution)
        cols = int(self.width / resolution)
        grid = np.zeros((rows, cols))
        
        for i in range(rows):
            for j in range(cols):
                x = j * resolution + resolution / 2
                y = i * resolution + resolution / 2
                if self.is_collision(x, y):
                    grid[i, j] = 1
        return grid, resolution

class AStarPlanner:
    def __init__(self, maze, resolution=0.3):
        self.maze = maze
        self.grid, self.resolution = maze.create_grid(resolution)
        self.rows, self.cols = self.grid.shape
        
    def heuristic(self, a, b):
        return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)
    
    def get_neighbors(self, pos):
        neighbors = []
        for di, dj in [(-1,0), (1,0), (0,-1), (0,1), (-1,-1), (-1,1), (1,-1), (1,1)]:
            ni, nj = pos[0] + di, pos[1] + dj
            if 0 <= ni < self.rows and 0 <= nj < self.cols and self.grid[ni, nj] == 0:
                neighbors.append((ni, nj))
        return neighbors
    
    def plan(self, start, goal):
        start_grid = (int(start[1] / self.resolution), int(start[0] / self.resolution))
        goal_grid = (int(goal[1] / self.resolution), int(goal[0] / self.resolution))
        
        if self.grid[start_grid] == 1 or self.grid[goal_grid] == 1:
            return []
        
        open_set = []
        heapq.heappush(open_set, (0, start_grid))
        came_from = {}
        g_score = {start_grid: 0}
        f_score = {start_grid: self.heuristic(start_grid, goal_grid)}
        
        while open_set:
            current = heapq.heappop(open_set)[1]
            
            if current == goal_grid:
                path = []
                while current in came_from:
                    x = current[1] * self.resolution + self.resolution / 2
                    y = current[0] * self.resolution + self.resolution / 2
                    path.append((x, y))
                    current = came_from[current]
                path.reverse()
                return path
            
            for neighbor in self.get_neighbors(current):
                tentative_g = g_score[current] + self.heuristic(current, neighbor)
                
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + self.heuristic(neighbor, goal_grid)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
        
        return []

class Controller:
    def __init__(self, robot, path, maze, max_speed=1.0):
        self.robot = robot
        self.path = path
        self.maze = maze
        self.max_speed = max_speed
        self.current_waypoint = 0
        self.sensor_range = 0.5
        self.goal_reached = False
        
    def get_control(self, dt):
        if self.current_waypoint >= len(self.path):
            return 0, 0
        
        target = self.path[self.current_waypoint]
        dx = target[0] - self.robot.x
        dy = target[1] - self.robot.y
        distance = np.sqrt(dx**2 + dy**2)
        
        if distance < 0.3:
            self.current_waypoint += 1
            if self.current_waypoint >= len(self.path):
                self.goal_reached = True
                return 0, 0
            target = self.path[self.current_waypoint]
            dx = target[0] - self.robot.x
            dy = target[1] - self.robot.y
        
        desired_theta = np.arctan2(dy, dx)
        angle_error = np.arctan2(np.sin(desired_theta - self.robot.theta),
                                 np.cos(desired_theta - self.robot.theta))
        
        obstacle_detected = False
        obstacle_angle = 0
        
        for angle_offset in np.linspace(-np.pi/3, np.pi/3, 5):
            sensor_angle = self.robot.theta + angle_offset
            check_x = self.robot.x + self.sensor_range * np.cos(sensor_angle)
            check_y = self.robot.y + self.sensor_range * np.sin(sensor_angle)
            
            if self.maze.is_collision(check_x, check_y, 0.15):
                obstacle_detected = True
                obstacle_angle = angle_offset
                break
        
        if obstacle_detected:
            v_left = 0.3
            v_right = 0.8 if obstacle_angle < 0 else 0.2
        else:
            v = self.max_speed * min(1.0, distance)
            omega = 3.0 * angle_error
            
            v_left = v - omega * self.robot.wheel_base / 2
            v_right = v + omega * self.robot.wheel_base / 2
            
            max_wheel_speed = 1.5
            v_left = np.clip(v_left, -max_wheel_speed, max_wheel_speed)
            v_right = np.clip(v_right, -max_wheel_speed, max_wheel_speed)
        
        return v_left, v_right

# Setup
maze = Maze(width=10, height=10)
start_pos = (1, 1)
goal_pos = (8.5, 8.5)

planner = AStarPlanner(maze, resolution=0.3)
path = planner.plan(start_pos, goal_pos)

if not path:
    path = [goal_pos]

robot = DifferentialDriveRobot(start_pos[0], start_pos[1], np.pi/4)
controller = Controller(robot, path, maze)

# Create figure
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(0, maze.width)
ax.set_ylim(0, maze.height)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_facecolor('#f0f0f0')

# Draw obstacles
for obs in maze.obstacles:
    rect = Rectangle(obs.get_xy(), obs.get_width(), obs.get_height(), 
                    facecolor='#404040', edgecolor='black', linewidth=2)
    ax.add_patch(rect)

# Draw path
if path:
    path_x, path_y = zip(*path)
    ax.plot(path_x, path_y, 'b--', linewidth=2, alpha=0.5, label='Planned Path')

# Markers
ax.plot(start_pos[0], start_pos[1], 'go', markersize=15, label='Start', zorder=10)
ax.plot(goal_pos[0], goal_pos[1], 'r*', markersize=20, label='Goal', zorder=10)

# Robot elements - rectangular chassis with two wheels
robot_width = 0.3
robot_length = 0.4
robot_body = Rectangle((0, 0), robot_length, robot_width, 
                       facecolor='#FF4444', edgecolor='darkred', 
                       linewidth=2, zorder=5)
ax.add_patch(robot_body)

# Two wheels on sides
wheel_width = 0.08
wheel_length = 0.15
left_wheel = Rectangle((0, 0), wheel_length, wheel_width, 
                       facecolor='#202020', edgecolor='black', linewidth=1, zorder=6)
right_wheel = Rectangle((0, 0), wheel_length, wheel_width, 
                        facecolor='#202020', edgecolor='black', linewidth=1, zorder=6)
ax.add_patch(left_wheel)
ax.add_patch(right_wheel)

# Direction indicator (front arrow)
robot_dir, = ax.plot([], [], 'yellow', linewidth=3, marker='>', markersize=8, zorder=7)
trail, = ax.plot([], [], 'r-', linewidth=1.5, alpha=0.4, label='Robot Trail')

title = ax.text(5, 10.5, 'Starting...', ha='center', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=10)

# Animation state
dt = 0.05
frame_count = [0]

def init():
    robot_dir.set_data([], [])
    trail.set_data([], [])
    return robot_body, left_wheel, right_wheel, robot_dir, trail, title

def animate(frame):
    # Update robot
    v_left, v_right = controller.get_control(dt)
    robot.update(v_left, v_right, dt)
    
    # Update robot body (rectangular chassis)
    # Create corners of rectangle centered at origin
    corners = np.array([
        [-robot_length/2, -robot_width/2],
        [robot_length/2, -robot_width/2],
        [robot_length/2, robot_width/2],
        [-robot_length/2, robot_width/2]
    ])
    
    # Rotation matrix
    cos_t = np.cos(robot.theta)
    sin_t = np.sin(robot.theta)
    rotation = np.array([[cos_t, -sin_t], [sin_t, cos_t]])
    
    # Rotate and translate
    rotated_corners = corners @ rotation.T
    rotated_corners[:, 0] += robot.x
    rotated_corners[:, 1] += robot.y
    
    robot_body.set_xy(rotated_corners[0])
    robot_body.set_angle(np.degrees(robot.theta))
    
    # Update wheels - positioned on left and right sides of chassis
    wheel_offset = robot_width / 2 + wheel_width / 2
    
    # Left wheel position
    left_x = robot.x - wheel_offset * sin_t
    left_y = robot.y + wheel_offset * cos_t
    left_wheel_corners = np.array([
        [-wheel_length/2, -wheel_width/2],
        [wheel_length/2, -wheel_width/2],
        [wheel_length/2, wheel_width/2],
        [-wheel_length/2, wheel_width/2]
    ])
    left_rotated = left_wheel_corners @ rotation.T
    left_rotated[:, 0] += left_x
    left_rotated[:, 1] += left_y
    left_wheel.set_xy(left_rotated[0])
    left_wheel.set_angle(np.degrees(robot.theta))
    
    # Right wheel position
    right_x = robot.x + wheel_offset * sin_t
    right_y = robot.y - wheel_offset * cos_t
    right_wheel_corners = np.array([
        [-wheel_length/2, -wheel_width/2],
        [wheel_length/2, -wheel_width/2],
        [wheel_length/2, wheel_width/2],
        [-wheel_length/2, wheel_width/2]
    ])
    right_rotated = right_wheel_corners @ rotation.T
    right_rotated[:, 0] += right_x
    right_rotated[:, 1] += right_y
    right_wheel.set_xy(right_rotated[0])
    right_wheel.set_angle(np.degrees(robot.theta))
    
    # Direction arrow at front of robot
    arrow_start_x = robot.x + (robot_length/4) * cos_t
    arrow_start_y = robot.y + (robot_length/4) * sin_t
    arrow_end_x = robot.x + (robot_length/2 + 0.15) * cos_t
    arrow_end_y = robot.y + (robot_length/2 + 0.15) * sin_t
    robot_dir.set_data([arrow_start_x, arrow_end_x], [arrow_start_y, arrow_end_y])
    
    # Trail
    if len(robot.path_history) > 1:
        trail_x, trail_y = zip(*robot.path_history)
        trail.set_data(trail_x, trail_y)
    
    # Update title
    if controller.goal_reached:
        title.set_text('🎉 GOAL REACHED! 🎉')
        title.set_color('green')
    else:
        dist_to_goal = np.sqrt((robot.x - goal_pos[0])**2 + (robot.y - goal_pos[1])**2)
        title.set_text(f'Distance to Goal: {dist_to_goal:.2f}m')
        title.set_color('black')
    
    frame_count[0] += 1
    
    return robot_body, left_wheel, right_wheel, robot_dir, trail, title

# Create animation - using cache_frame_data=False to ensure updates
anim = FuncAnimation(fig, animate, init_func=init, frames=2000, 
                    interval=30, blit=True, repeat=False, cache_frame_data=False)

from matplotlib.animation import FFMpegWriter
writer = FFMpegWriter(fps=30)
anim.save('robot_navigation_updated.mp4', writer=writer)

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

C:\Users\Abirami\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\matplotlib\animation.py:908: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you output the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(
C:\Users\Abirami\AppData\Local\Temp\ipykernel_42412\2087257839.py:349: UserWarning: Glyph 127881 (\N{PARTY POPPER}) missing from font(s) DejaVu Sans.
  anim.save('robot_navigation_updated.mp4', writer=writer)


In [6]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, Rectangle
from matplotlib.animation import FuncAnimation
import heapq

class DifferentialDriveRobot:
    def __init__(self, x, y, theta, wheel_base=0.3):
        self.x = x
        self.y = y
        self.theta = theta
        self.wheel_base = wheel_base
        self.path_history = [(x, y)]
        
    def update(self, v_left, v_right, dt):
        v = (v_right + v_left) / 2
        omega = (v_right - v_left) / self.wheel_base
        
        if abs(omega) < 1e-6:
            self.x += v * np.cos(self.theta) * dt
            self.y += v * np.sin(self.theta) * dt
        else:
            R = v / omega
            self.x += R * (np.sin(self.theta + omega * dt) - np.sin(self.theta))
            self.y += -R * (np.cos(self.theta + omega * dt) - np.cos(self.theta))
            self.theta += omega * dt
        
        self.theta = np.arctan2(np.sin(self.theta), np.cos(self.theta))
        self.path_history.append((self.x, self.y))

class DynamicObstacle:
    def __init__(self, x, y, vx, vy, radius=0.2):
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.radius = radius
        
    def update(self, dt, maze_width, maze_height):
        """Update obstacle position and bounce off walls"""
        self.x += self.vx * dt
        self.y += self.vy * dt
        
        # Bounce off walls
        if self.x - self.radius < 0.5 or self.x + self.radius > maze_width - 0.5:
            self.vx = -self.vx
            self.x = np.clip(self.x, 0.5 + self.radius, maze_width - 0.5 - self.radius)
        
        if self.y - self.radius < 0.5 or self.y + self.radius > maze_height - 0.5:
            self.vy = -self.vy
            self.y = np.clip(self.y, 0.5 + self.radius, maze_height - 0.5 - self.radius)

class Maze:
    def __init__(self, width=10, height=10):
        self.width = width
        self.height = height
        self.obstacles = []
        self.dynamic_obstacles = []
        self.create_maze()
        self.create_dynamic_obstacles()
        
    def create_maze(self):
        self.obstacles.append(Rectangle((0, 0), self.width, 0.2))
        self.obstacles.append(Rectangle((0, self.height-0.2), self.width, 0.2))
        self.obstacles.append(Rectangle((0, 0), 0.2, self.height))
        self.obstacles.append(Rectangle((self.width-0.2, 0), 0.2, self.height))
        
        self.obstacles.append(Rectangle((2, 2), 3, 0.3))
        self.obstacles.append(Rectangle((6, 4), 0.3, 3))
        self.obstacles.append(Rectangle((3, 6), 2, 0.3))
        self.obstacles.append(Rectangle((7, 1), 2, 0.3))
        self.obstacles.append(Rectangle((4, 8), 0.3, 1.5))
    
    def create_dynamic_obstacles(self):
        """Create moving obstacles"""
        self.dynamic_obstacles.append(DynamicObstacle(3.5, 4.0, 0.4, 0.3))
        self.dynamic_obstacles.append(DynamicObstacle(7.0, 7.0, -0.3, 0.4))
        self.dynamic_obstacles.append(DynamicObstacle(5.0, 2.5, 0.35, -0.35))
        self.dynamic_obstacles.append(DynamicObstacle(2.5, 8.0, 0.3, -0.25))
    
    def update_dynamic_obstacles(self, dt):
        """Update all dynamic obstacles"""
        for obs in self.dynamic_obstacles:
            obs.update(dt, self.width, self.height)
        
    def is_collision(self, x, y, robot_radius=0.2):
        for obs in self.obstacles:
            ox, oy = obs.get_xy()
            ow, oh = obs.get_width(), obs.get_height()
            if (ox - robot_radius <= x <= ox + ow + robot_radius and
                oy - robot_radius <= y <= oy + oh + robot_radius):
                return True
        return False
    
    def create_grid(self, resolution=0.3):
        rows = int(self.height / resolution)
        cols = int(self.width / resolution)
        grid = np.zeros((rows, cols))
        
        for i in range(rows):
            for j in range(cols):
                x = j * resolution + resolution / 2
                y = i * resolution + resolution / 2
                if self.is_collision(x, y):
                    grid[i, j] = 1
        return grid, resolution

class AStarPlanner:
    def __init__(self, maze, resolution=0.3):
        self.maze = maze
        self.grid, self.resolution = maze.create_grid(resolution)
        self.rows, self.cols = self.grid.shape
        
    def heuristic(self, a, b):
        return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)
    
    def get_neighbors(self, pos):
        neighbors = []
        for di, dj in [(-1,0), (1,0), (0,-1), (0,1), (-1,-1), (-1,1), (1,-1), (1,1)]:
            ni, nj = pos[0] + di, pos[1] + dj
            if 0 <= ni < self.rows and 0 <= nj < self.cols and self.grid[ni, nj] == 0:
                neighbors.append((ni, nj))
        return neighbors
    
    def plan(self, start, goal):
        start_grid = (int(start[1] / self.resolution), int(start[0] / self.resolution))
        goal_grid = (int(goal[1] / self.resolution), int(goal[0] / self.resolution))
        
        if self.grid[start_grid] == 1 or self.grid[goal_grid] == 1:
            return []
        
        open_set = []
        heapq.heappush(open_set, (0, start_grid))
        came_from = {}
        g_score = {start_grid: 0}
        f_score = {start_grid: self.heuristic(start_grid, goal_grid)}
        
        while open_set:
            current = heapq.heappop(open_set)[1]
            
            if current == goal_grid:
                path = []
                while current in came_from:
                    x = current[1] * self.resolution + self.resolution / 2
                    y = current[0] * self.resolution + self.resolution / 2
                    path.append((x, y))
                    current = came_from[current]
                path.reverse()
                return path
            
            for neighbor in self.get_neighbors(current):
                tentative_g = g_score[current] + self.heuristic(current, neighbor)
                
                if neighbor not in g_score or tentative_g < g_score[neighbor]:
                    came_from[neighbor] = current
                    g_score[neighbor] = tentative_g
                    f_score[neighbor] = tentative_g + self.heuristic(neighbor, goal_grid)
                    heapq.heappush(open_set, (f_score[neighbor], neighbor))
        
        return []

class Controller:
    def __init__(self, robot, path, maze, max_speed=1.0):
        self.robot = robot
        self.path = path
        self.maze = maze
        self.max_speed = max_speed
        self.current_waypoint = 0
        self.sensor_range = 0.6
        self.goal_reached = False
        self.dynamic_avoidance_active = False
        
    def get_control(self, dt):
        if self.current_waypoint >= len(self.path):
            return 0, 0
        
        target = self.path[self.current_waypoint]
        dx = target[0] - self.robot.x
        dy = target[1] - self.robot.y
        distance = np.sqrt(dx**2 + dy**2)
        
        if distance < 0.3:
            self.current_waypoint += 1
            if self.current_waypoint >= len(self.path):
                self.goal_reached = True
                return 0, 0
            target = self.path[self.current_waypoint]
            dx = target[0] - self.robot.x
            dy = target[1] - self.robot.y
        
        desired_theta = np.arctan2(dy, dx)
        angle_error = np.arctan2(np.sin(desired_theta - self.robot.theta),
                                 np.cos(desired_theta - self.robot.theta))
        
        # Enhanced obstacle detection for both static and dynamic obstacles
        obstacle_detected = False
        obstacle_angle = 0
        min_distance = float('inf')
        
        # Check in multiple directions
        for angle_offset in np.linspace(-np.pi/2, np.pi/2, 9):
            sensor_angle = self.robot.theta + angle_offset
            check_x = self.robot.x + self.sensor_range * np.cos(sensor_angle)
            check_y = self.robot.y + self.sensor_range * np.sin(sensor_angle)
            
            if self.maze.is_collision(check_x, check_y, 0.15):
                obstacle_detected = True
                obstacle_angle = angle_offset
                
                # Calculate distance to obstacle
                dist = np.sqrt((check_x - self.robot.x)**2 + (check_y - self.robot.y)**2)
                if dist < min_distance:
                    min_distance = dist
                break
        
        # Dynamic obstacle avoidance - check for nearby moving obstacles
        closest_dynamic_obs = None
        closest_dynamic_dist = float('inf')
        for dyn_obs in self.maze.dynamic_obstacles:
            dist = np.sqrt((self.robot.x - dyn_obs.x)**2 + (self.robot.y - dyn_obs.y)**2)
            if dist < closest_dynamic_dist and dist < 1.0:  # Detection range
                closest_dynamic_dist = dist
                closest_dynamic_obs = dyn_obs
        
        self.dynamic_avoidance_active = False
        
        if obstacle_detected and min_distance < 0.4:
            # Reactive avoidance for close obstacles
            v_left = 0.3
            v_right = 0.8 if obstacle_angle < 0 else 0.2
            self.dynamic_avoidance_active = True
        elif closest_dynamic_obs is not None and closest_dynamic_dist < 0.7:
            # Predict dynamic obstacle position and avoid
            predict_time = 0.5
            pred_x = closest_dynamic_obs.x + closest_dynamic_obs.vx * predict_time
            pred_y = closest_dynamic_obs.y + closest_dynamic_obs.vy * predict_time
            
            # Calculate avoidance direction
            avoid_angle = np.arctan2(pred_y - self.robot.y, pred_x - self.robot.x)
            angle_to_obs = np.arctan2(np.sin(avoid_angle - self.robot.theta),
                                      np.cos(avoid_angle - self.robot.theta))
            
            # Steer away from predicted position
            v = 0.5
            omega = -4.0 * angle_to_obs  # Turn away from obstacle
            
            v_left = v - omega * self.robot.wheel_base / 2
            v_right = v + omega * self.robot.wheel_base / 2
            
            max_wheel_speed = 1.5
            v_left = np.clip(v_left, -max_wheel_speed, max_wheel_speed)
            v_right = np.clip(v_right, -max_wheel_speed, max_wheel_speed)
            self.dynamic_avoidance_active = True
        else:
            # Pure pursuit
            v = self.max_speed * min(1.0, distance)
            omega = 3.0 * angle_error
            
            v_left = v - omega * self.robot.wheel_base / 2
            v_right = v + omega * self.robot.wheel_base / 2
            
            max_wheel_speed = 1.5
            v_left = np.clip(v_left, -max_wheel_speed, max_wheel_speed)
            v_right = np.clip(v_right, -max_wheel_speed, max_wheel_speed)
        
        return v_left, v_right

# Setup
maze = Maze(width=10, height=10)
start_pos = (1, 1)
goal_pos = (8.5, 8.5)

planner = AStarPlanner(maze, resolution=0.3)
path = planner.plan(start_pos, goal_pos)

if not path:
    path = [goal_pos]

robot = DifferentialDriveRobot(start_pos[0], start_pos[1], np.pi/4)
controller = Controller(robot, path, maze)

# Create figure
fig, ax = plt.subplots(figsize=(10, 10))
ax.set_xlim(0, maze.width)
ax.set_ylim(0, maze.height)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_facecolor('#f0f0f0')

# Draw obstacles
for obs in maze.obstacles:
    rect = Rectangle(obs.get_xy(), obs.get_width(), obs.get_height(), 
                    facecolor='#404040', edgecolor='black', linewidth=2)
    ax.add_patch(rect)

# Draw path
if path:
    path_x, path_y = zip(*path)
    ax.plot(path_x, path_y, 'b--', linewidth=2, alpha=0.5, label='Planned Path')

# Markers
ax.plot(start_pos[0], start_pos[1], 'go', markersize=15, label='Start', zorder=10)
ax.plot(goal_pos[0], goal_pos[1], 'r*', markersize=20, label='Goal', zorder=10)

# Draw dynamic obstacles
dynamic_obs_circles = []
for dyn_obs in maze.dynamic_obstacles:
    circle = Circle((dyn_obs.x, dyn_obs.y), dyn_obs.radius, 
                   color='orange', ec='darkorange', linewidth=2, 
                   alpha=0.7, zorder=4, label='Dynamic Obstacle' if len(dynamic_obs_circles) == 0 else '')
    ax.add_patch(circle)
    dynamic_obs_circles.append(circle)

# Robot elements - rectangular chassis with two wheels
robot_width = 0.3
robot_length = 0.4
robot_body = Rectangle((0, 0), robot_length, robot_width, 
                       facecolor='#FF4444', edgecolor='darkred', 
                       linewidth=2, zorder=5)
ax.add_patch(robot_body)

# Two wheels on sides
wheel_width = 0.08
wheel_length = 0.15
left_wheel = Rectangle((0, 0), wheel_length, wheel_width, 
                       facecolor='#202020', edgecolor='black', linewidth=1, zorder=6)
right_wheel = Rectangle((0, 0), wheel_length, wheel_width, 
                        facecolor='#202020', edgecolor='black', linewidth=1, zorder=6)
ax.add_patch(left_wheel)
ax.add_patch(right_wheel)

# Direction indicator (front arrow)
robot_dir, = ax.plot([], [], 'yellow', linewidth=3, marker='>', markersize=8, zorder=7)
trail, = ax.plot([], [], 'r-', linewidth=1.5, alpha=0.4, label='Robot Trail')

title = ax.text(5, 10.5, 'Starting...', ha='center', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=10)

# Animation state
dt = 0.05
frame_count = [0]

def init():
    robot_dir.set_data([], [])
    trail.set_data([], [])
    return robot_body, left_wheel, right_wheel, robot_dir, trail, title, *dynamic_obs_circles

def animate(frame):
    # Update dynamic obstacles
    maze.update_dynamic_obstacles(dt)
    
    # Update dynamic obstacle visuals
    for i, dyn_obs in enumerate(maze.dynamic_obstacles):
        dynamic_obs_circles[i].center = (dyn_obs.x, dyn_obs.y)
    
    # Update robot
    v_left, v_right = controller.get_control(dt)
    robot.update(v_left, v_right, dt)
    
    # Update robot body (rectangular chassis)
    # Create corners of rectangle centered at origin
    corners = np.array([
        [-robot_length/2, -robot_width/2],
        [robot_length/2, -robot_width/2],
        [robot_length/2, robot_width/2],
        [-robot_length/2, robot_width/2]
    ])
    
    # Rotation matrix
    cos_t = np.cos(robot.theta)
    sin_t = np.sin(robot.theta)
    rotation = np.array([[cos_t, -sin_t], [sin_t, cos_t]])
    
    # Rotate and translate
    rotated_corners = corners @ rotation.T
    rotated_corners[:, 0] += robot.x
    rotated_corners[:, 1] += robot.y
    
    robot_body.set_xy(rotated_corners[0])
    robot_body.set_angle(np.degrees(robot.theta))
    
    # Update wheels - positioned on left and right sides of chassis
    wheel_offset = robot_width / 2 + wheel_width / 2
    
    # Left wheel position
    left_x = robot.x - wheel_offset * sin_t
    left_y = robot.y + wheel_offset * cos_t
    left_wheel_corners = np.array([
        [-wheel_length/2, -wheel_width/2],
        [wheel_length/2, -wheel_width/2],
        [wheel_length/2, wheel_width/2],
        [-wheel_length/2, wheel_width/2]
    ])
    left_rotated = left_wheel_corners @ rotation.T
    left_rotated[:, 0] += left_x
    left_rotated[:, 1] += left_y
    left_wheel.set_xy(left_rotated[0])
    left_wheel.set_angle(np.degrees(robot.theta))
    
    # Right wheel position
    right_x = robot.x + wheel_offset * sin_t
    right_y = robot.y - wheel_offset * cos_t
    right_wheel_corners = np.array([
        [-wheel_length/2, -wheel_width/2],
        [wheel_length/2, -wheel_width/2],
        [wheel_length/2, wheel_width/2],
        [-wheel_length/2, wheel_width/2]
    ])
    right_rotated = right_wheel_corners @ rotation.T
    right_rotated[:, 0] += right_x
    right_rotated[:, 1] += right_y
    right_wheel.set_xy(right_rotated[0])
    right_wheel.set_angle(np.degrees(robot.theta))
    
    # Direction arrow at front of robot
    arrow_start_x = robot.x + (robot_length/4) * cos_t
    arrow_start_y = robot.y + (robot_length/4) * sin_t
    arrow_end_x = robot.x + (robot_length/2 + 0.15) * cos_t
    arrow_end_y = robot.y + (robot_length/2 + 0.15) * sin_t
    robot_dir.set_data([arrow_start_x, arrow_end_x], [arrow_start_y, arrow_end_y])
    
    # Trail
    if len(robot.path_history) > 1:
        trail_x, trail_y = zip(*robot.path_history)
        trail.set_data(trail_x, trail_y)
    
    # Update title
    if controller.goal_reached:
        title.set_text('🎉 GOAL REACHED! 🎉')
        title.set_color('green')
    else:
        dist_to_goal = np.sqrt((robot.x - goal_pos[0])**2 + (robot.y - goal_pos[1])**2)
        status = "AVOIDING DYNAMIC OBSTACLE" if controller.dynamic_avoidance_active else "Following Path"
        title.set_text(f'{status} | Distance to Goal: {dist_to_goal:.2f}m')
        title.set_color('red' if controller.dynamic_avoidance_active else 'black')
    
    frame_count[0] += 1
    
    return robot_body, left_wheel, right_wheel, robot_dir, trail, title, *dynamic_obs_circles

# Create animation - using cache_frame_data=False to ensure updates
anim = FuncAnimation(fig, animate, init_func=init, frames=2000, 
                    interval=30, blit=True, repeat=False, cache_frame_data=False)

from matplotlib.animation import FFMpegWriter
writer = FFMpegWriter(fps=30)
anim.save('robot_navigation_updated_v2.mp4', writer=writer)

plt.tight_layout()
plt.show()

<IPython.core.display.Javascript object>

C:\Users\Abirami\AppData\Local\Temp\ipykernel_42412\2924535042.py:447: UserWarning: Glyph 127881 (\N{PARTY POPPER}) missing from font(s) DejaVu Sans.
  anim.save('robot_navigation_updated_v2.mp4', writer=writer)


In [7]:
# --- Kinematic Differential-drive semicircle animation with plots ---
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib.patches import Rectangle, Circle
from matplotlib.transforms import Affine2D
from IPython.display import Video
import math

# ---------- Parameters (change these to experiment) ----------
wheel_base = 0.6        # distance between wheel centers (m) (track)
body_length = 0.8       # body length (m) (front to back)
body_width  = wheel_base + 0.08  # body width (slightly bigger than track)
wheel_radius = 0.09     # wheel radius (m)
dt = 0.05               # simulation timestep (s)

# Semicircle path parameters (radius of curvature and robot turn rate)
R_c = 2.0               # radius of the semicircle path (m)
omega = 0.4             # angular rate around curvature center (rad/s)

# Derived wheel velocities that produce the chosen curvature:
# v_r = omega * (R_c + b/2), v_l = omega * (R_c - b/2)
v_r = omega * (R_c + wheel_base/2.0)
v_l = omega * (R_c - wheel_base/2.0)

# total time to sweep pi radians (a semicircle) and number of steps
T_total = math.pi / omega
steps = int(np.ceil(T_total / dt)) + 1

print(f"Computed wheel speeds for semicircle: v_left = {v_l:.4f} m/s, v_right = {v_r:.4f} m/s")
print(f"Total time ≈ {T_total:.2f} s, frames = {steps}")

# ---------- Simulate poses and velocities ----------
poses = np.zeros((steps, 3))   # columns x, y, theta
# Store velocities for plotting
v_body = np.zeros(steps)       # body frame forward velocity
omega_body = np.zeros(steps)   # body frame angular velocity
v_x_inertial = np.zeros(steps) # x-component in inertial frame
v_y_inertial = np.zeros(steps) # y-component in inertial frame
omega_l = np.zeros(steps)      # left wheel angular velocity (rad/s)
omega_r = np.zeros(steps)      # right wheel angular velocity (rad/s)
v_l_linear = np.zeros(steps)   # left wheel linear velocity (m/s)
v_r_linear = np.zeros(steps)   # right wheel linear velocity (m/s)
time_array = np.zeros(steps)   # time array for plotting

x = 0.0; y = 0.0; theta = 0.0  # initial pose (facing +x)
for i in range(steps):
    poses[i, :] = [x, y, theta]
    time_array[i] = i * dt
    
    # Compute velocities
    v = (v_r + v_l) / 2.0
    w = (v_r - v_l) / wheel_base
    
    # Body frame velocity
    v_body[i] = v
    omega_body[i] = w
    
    # Inertial frame velocity components
    v_x_inertial[i] = v * math.cos(theta)
    v_y_inertial[i] = v * math.sin(theta)
    
    # Wheel angular velocities (omega = v / r)
    omega_l[i] = v_l / wheel_radius
    omega_r[i] = v_r / wheel_radius
    
    # Wheel linear velocities
    v_l_linear[i] = v_l
    v_r_linear[i] = v_r
    
    # Update pose
    x += v * math.cos(theta) * dt
    y += v * math.sin(theta) * dt
    theta += w * dt

# ---------- Set up plot limits so entire motion is visible ----------
xmin, xmax = poses[:,0].min(), poses[:,0].max()
ymin, ymax = poses[:,1].min(), poses[:,1].max()
pad = max(body_length, R_c) * 0.6 + 0.2
xlim = (xmin - pad, xmax + pad)
ylim = (ymin - pad, ymax + pad)

# ---------- Create figure and static elements ----------
fig, ax = plt.subplots(figsize=(7,7))
ax.set_aspect('equal', 'box')
ax.set_xlim(*xlim)
ax.set_ylim(*ylim)
ax.set_title("Differential-drive robot — semicircle motion")

# full planned path (faint dashed) and traveled path (colored)
ax.plot(poses[:,0], poses[:,1], linestyle='--', color='lightgray', label='planned path')
path_line, = ax.plot([], [], 'r-', lw=2, label='traveled path')

# A rectangle representing the body, constructed centered on origin in robot frame:
body = Rectangle((-body_length/2, -body_width/2), body_length, body_width,
                 edgecolor='tab:blue', facecolor='none', lw=2, zorder=4)
ax.add_patch(body)

# Wheels (initial positions will be set each frame)
wheel_left = Circle((0,  wheel_base/2), wheel_radius, color='k', zorder=5)
wheel_right= Circle((0, -wheel_base/2), wheel_radius, color='k', zorder=5)
ax.add_patch(wheel_left)
ax.add_patch(wheel_right)

# Text: velocities (top-left)
vel_text = ax.text(0.02, 0.98, '', transform=ax.transAxes, fontsize=11, va='top')

# Text boxes next to wheels to display their speeds (will move with wheels)
wheelL_label = ax.text(0,0,'', fontsize=9, va='center', ha='left',
                       bbox=dict(facecolor='white', alpha=0.7), zorder=6)
wheelR_label = ax.text(0,0,'', fontsize=9, va='center', ha='right',
                       bbox=dict(facecolor='white', alpha=0.7), zorder=6)

# Heading line (from robot center forward)
heading_line, = ax.plot([], [], color='g', lw=2, zorder=6)

# small helper to transform a point from robot frame (xr, yr) -> world
def robot_to_world(xr, yr, xw, yw, th):
    X = xw + xr * math.cos(th) - yr * math.sin(th)
    Y = yw + xr * math.sin(th) + yr * math.cos(th)
    return X, Y

# ---------- Animation functions ----------
def init():
    path_line.set_data([], [])
    vel_text.set_text('')
    heading_line.set_data([], [])
    body.set_transform(Affine2D().translate(0,0) + ax.transData)
    wheel_left.center = robot_to_world(0, wheel_base/2, 0, 0, 0)
    wheel_right.center= robot_to_world(0, -wheel_base/2, 0, 0, 0)
    wheelL_label.set_position((0,0))
    wheelR_label.set_position((0,0))
    wheelL_label.set_text('')
    wheelR_label.set_text('')
    return path_line, body, wheel_left, wheel_right, heading_line, vel_text, wheelL_label, wheelR_label

def update(i):
    ax.set_xlabel("X position (m) [forward direction]")
    ax.set_ylabel("Y position (m) [lateral direction]")

    x, y, th = poses[i]
    path_line.set_data(poses[:i+1,0], poses[:i+1,1])

    tr = Affine2D().rotate(th).translate(x, y) + ax.transData
    body.set_transform(tr)

    xL, yL = robot_to_world(0,  wheel_base/2, x, y, th)
    xR, yR = robot_to_world(0, -wheel_base/2, x, y, th)
    wheel_left.center = (xL, yL)
    wheel_right.center = (xR, yR)

    head_len = body_length * 0.6
    hx, hy = x + head_len*math.cos(th), y + head_len*math.sin(th)
    heading_line.set_data([x, hx], [y, hy])

    v = (v_r + v_l) / 2.0
    w = (v_r - v_l) / wheel_base
    vel_text.set_text(
        f"t = {i*dt:.2f} s\nv_left = {v_l:.3f} m/s\nv_right = {v_r:.3f} m/s\nv = {v:.3f} m/s\nomega = {w:.3f} rad/s"
    )

    off = 0.06
    lx_off = xL + off * math.cos(th)
    ly_off = yL + off * math.sin(th)
    rx_off = xR + off * math.cos(th)
    ry_off = yR + off * math.sin(th)

    wheelL_label.set_position((lx_off, ly_off))
    wheelR_label.set_position((rx_off, ry_off))
    wheelL_label.set_text(f"{v_l:.3f} m/s")
    wheelR_label.set_text(f"{v_r:.3f} m/s")

    return path_line, body, wheel_left, wheel_right, heading_line, vel_text, wheelL_label, wheelR_label

# ---------- Make and save animation ----------
ani = FuncAnimation(fig, update, frames=len(poses), init_func=init,
                    blit=False, interval=dt*1000)

fps = int(round(1.0 / dt))
writer = FFMpegWriter(fps=fps)
out_filename = "diff_drive_semicircle.mp4"
ani.save(out_filename, writer=writer)
plt.close(fig)

print(f"Saved video -> {out_filename}")

# ---------- Create velocity plots ----------
fig2, axes = plt.subplots(2, 2, figsize=(14, 10))
fig2.suptitle('Differential Drive Robot Velocity Analysis', fontsize=16, fontweight='bold')

# Plot 1: Robot Body Frame Velocity
axes[0, 0].plot(time_array, v_body, 'b-', linewidth=2, label='Linear velocity (v)')
axes[0, 0].plot(time_array, omega_body, 'r-', linewidth=2, label='Angular velocity (ω)')
axes[0, 0].set_xlabel('Time (s)', fontsize=12)
axes[0, 0].set_ylabel('Velocity', fontsize=12)
axes[0, 0].set_title('1. Robot Body Frame Velocity', fontsize=13, fontweight='bold')
axes[0, 0].legend(loc='best', fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Velocity Components in Inertial Frame
axes[0, 1].plot(time_array, v_x_inertial, 'g-', linewidth=2, label='vₓ (x-component)')
axes[0, 1].plot(time_array, v_y_inertial, 'm-', linewidth=2, label='vᵧ (y-component)')
axes[0, 1].set_xlabel('Time (s)', fontsize=12)
axes[0, 1].set_ylabel('Velocity (m/s)', fontsize=12)
axes[0, 1].set_title('2. Velocity Components in Inertial Frame', fontsize=13, fontweight='bold')
axes[0, 1].legend(loc='best', fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Left and Right Wheel Angular Velocities
axes[1, 0].plot(time_array, omega_l, 'c-', linewidth=2, label='Left wheel (ωₗ)')
axes[1, 0].plot(time_array, omega_r, 'orange', linewidth=2, label='Right wheel (ωᵣ)')
axes[1, 0].set_xlabel('Time (s)', fontsize=12)
axes[1, 0].set_ylabel('Angular Velocity (rad/s)', fontsize=12)
axes[1, 0].set_title('3. Left and Right Wheel Angular Velocities', fontsize=13, fontweight='bold')
axes[1, 0].legend(loc='best', fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Wheel Linear Velocities
axes[1, 1].plot(time_array, v_l_linear, 'c-', linewidth=2, label='Left wheel (vₗ)')
axes[1, 1].plot(time_array, v_r_linear, 'orange', linewidth=2, label='Right wheel (vᵣ)')
axes[1, 1].set_xlabel('Time (s)', fontsize=12)
axes[1, 1].set_ylabel('Linear Velocity (m/s)', fontsize=12)
axes[1, 1].set_title('4. Wheel Linear Velocities', fontsize=13, fontweight='bold')
axes[1, 1].legend(loc='best', fontsize=10)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plot_filename = "velocity_plots.png"
plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
print(f"Saved plots -> {plot_filename}")
plt.show()

# Display the video inline in Colab
Video(out_filename, embed=True)

Computed wheel speeds for semicircle: v_left = 0.6800 m/s, v_right = 0.9200 m/s
Total time ≈ 7.85 s, frames = 159


<IPython.core.display.Javascript object>

AttributeError: 'NoneType' object has no attribute 'remove_callback'

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from matplotlib.patches import Rectangle, Circle
from matplotlib.transforms import Affine2D
from IPython.display import Video

# --- Robot Physical Parameters ---
R = 0.1         # Wheel radius (m)
L = 0.5         # Distance between wheels (m)
m = 15.0        # Robot mass (kg)
I = 1.0         # Robot moment of inertia (kg·m²)
b_v = 2.0       # Linear damping coefficient (N·s/m)
b_omega = 0.3   # Angular damping coefficient (N·m·s/rad)

# --- Simulation Parameters ---
dt = 0.02       # Time step (s)
T_total = 30    # Total simulation time (s)
num_steps = int(T_total / dt)

print(f"Simulating {T_total}s motion with {num_steps} steps...")

# --- Figure-8 Path Parameters ---
# The robot will trace a figure-8 pattern
A = 3.0         # Amplitude of figure-8 (m)
omega_path = 2 * np.pi / T_total  # Angular frequency for one complete figure-8

# --- Generate reference trajectory (figure-8) ---
t_ref = np.linspace(0, T_total, num_steps)
x_ref = A * np.sin(omega_path * t_ref)
y_ref = A * np.sin(2 * omega_path * t_ref) / 2
theta_ref = np.arctan2(
    A * omega_path * 2 * np.cos(2 * omega_path * t_ref),
    A * omega_path * np.cos(omega_path * t_ref)
)
v_ref = A * omega_path * np.sqrt(
    np.cos(omega_path * t_ref)**2 + 
    np.cos(2 * omega_path * t_ref)**2
)

# --- Controller gains (PD controller for trajectory tracking) ---
Kp_v = 5.0      # Proportional gain for velocity
Kp_omega = 8.0  # Proportional gain for angular velocity
Kd_v = 2.0      # Derivative gain for velocity
Kd_omega = 1.0  # Derivative gain for angular velocity

# --- Storage arrays ---
time_array = np.zeros(num_steps)
x_array = np.zeros(num_steps)
y_array = np.zeros(num_steps)
theta_array = np.zeros(num_steps)
v_array = np.zeros(num_steps)
omega_array = np.zeros(num_steps)
a_array = np.zeros(num_steps)
alpha_array = np.zeros(num_steps)
force_array = np.zeros(num_steps)
torque_array = np.zeros(num_steps)
torque_left = np.zeros(num_steps)
torque_right = np.zeros(num_steps)
wheel_omega_left = np.zeros(num_steps)
wheel_omega_right = np.zeros(num_steps)
v_left_array = np.zeros(num_steps)
v_right_array = np.zeros(num_steps)
curvature_array = np.zeros(num_steps)
path_error = np.zeros(num_steps)
heading_error = np.zeros(num_steps)
v_x_array = np.zeros(num_steps)
v_y_array = np.zeros(num_steps)
power_left = np.zeros(num_steps)
power_right = np.zeros(num_steps)
total_power = np.zeros(num_steps)

# --- Initial State ---
x = x_ref[0]
y = y_ref[0]
theta = theta_ref[0]
v = 0.0
omega = 0.0
v_prev = 0.0
omega_prev = 0.0

# --- Simulation Loop with PD Control ---
for i in range(num_steps):
    time_array[i] = i * dt
    x_array[i] = x
    y_array[i] = y
    theta_array[i] = theta
    v_array[i] = v
    omega_array[i] = omega
    
    # Position error
    dx = x_ref[i] - x
    dy = y_ref[i] - y
    path_error[i] = np.sqrt(dx**2 + dy**2)
    
    # Heading error
    heading_error[i] = theta_ref[i] - theta
    # Normalize to [-pi, pi]
    while heading_error[i] > np.pi:
        heading_error[i] -= 2*np.pi
    while heading_error[i] < -np.pi:
        heading_error[i] += 2*np.pi
    
    # Desired velocities from reference + error correction
    v_desired = v_ref[i] + Kp_v * (dx * np.cos(theta) + dy * np.sin(theta))
    omega_desired = Kp_omega * heading_error[i]
    
    # Compute control inputs with derivative terms
    v_error = v_desired - v
    omega_error = omega_desired - omega
    
    # Desired wheel velocities
    v_left = v_desired - (L/2) * omega_desired
    v_right = v_desired + (L/2) * omega_desired
    
    v_left_array[i] = v_left
    v_right_array[i] = v_right
    
    # Wheel angular velocities
    wheel_omega_left[i] = v_left / R
    wheel_omega_right[i] = v_right / R
    
    # Compute accelerations
    a = (v_desired - v) / dt if dt > 0 else 0
    alpha = (omega_desired - omega) / dt if dt > 0 else 0
    a_array[i] = a
    alpha_array[i] = alpha
    
    # Dynamics: F = ma + damping
    F_net = m * a + b_v * v
    tau_net = I * alpha + b_omega * omega
    
    force_array[i] = F_net
    torque_array[i] = tau_net
    
    # Wheel torques (distributed from net force and torque)
    # For differential drive: τ_L + τ_R = F*R, τ_R - τ_L = τ_net*2/L
    torque_left[i] = (F_net * R - tau_net * 2 / L) / 2
    torque_right[i] = (F_net * R + tau_net * 2 / L) / 2
    
    # Power = torque * angular_velocity
    power_left[i] = abs(torque_left[i] * wheel_omega_left[i])
    power_right[i] = abs(torque_right[i] * wheel_omega_right[i])
    total_power[i] = power_left[i] + power_right[i]
    
    # Curvature
    if v > 0.01:
        curvature_array[i] = omega / v
    else:
        curvature_array[i] = 0
    
    # Velocity components in inertial frame
    v_x_array[i] = v * np.cos(theta)
    v_y_array[i] = v * np.sin(theta)
    
    # Update state (simple Euler integration)
    v += a * dt
    omega += alpha * dt
    x += v * np.cos(theta) * dt
    y += v * np.sin(theta) * dt
    theta += omega * dt

print("✅ Simulation complete!")

# --- Create Animation ---
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_aspect('equal')
margin = 1.0
x_min, x_max = x_ref.min() - margin, x_ref.max() + margin
y_min, y_max = y_ref.min() - margin, y_ref.max() + margin
ax.set_xlim(x_min, x_max)
ax.set_ylim(y_min, y_max)
ax.set_xlabel('X Position (m)', fontsize=12)
ax.set_ylabel('Y Position (m)', fontsize=12)
ax.set_title('Differential Drive Robot - Figure-8 Dynamic Trajectory Tracking', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Reference path
ax.plot(x_ref, y_ref, 'g--', linewidth=2, label='Reference path', alpha=0.6)

# Traveled path
path_line, = ax.plot([], [], 'b-', linewidth=2.5, label='Actual path', alpha=0.8)

# Robot body
body_width = L * 0.6
body_length = L * 0.8
body = Rectangle((-body_length/2, -body_width/2), body_length, body_width, 
                 fc='royalblue', ec='darkblue', linewidth=2, zorder=4)
ax.add_patch(body)

# Wheels
wheel_l = Circle((0, 0), R, fc='black', ec='gray', linewidth=1, zorder=5)
wheel_r = Circle((0, 0), R, fc='black', ec='gray', linewidth=1, zorder=5)
ax.add_patch(wheel_l)
ax.add_patch(wheel_r)

# Heading arrow
heading_line, = ax.plot([], [], 'r-', linewidth=3, zorder=6)

# Info text
info_text = ax.text(0.02, 0.98, '', transform=ax.transAxes, fontsize=10, 
                    verticalalignment='top', fontfamily='monospace',
                    bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))

ax.legend(loc='upper right', fontsize=10)

def init():
    path_line.set_data([], [])
    heading_line.set_data([], [])
    info_text.set_text('')
    return body, wheel_l, wheel_r, path_line, heading_line, info_text

def update(frame):
    curr_x = x_array[frame]
    curr_y = y_array[frame]
    curr_theta = theta_array[frame]
    curr_v = v_array[frame]
    curr_omega = omega_array[frame]
    
    # Update traveled path
    path_line.set_data(x_array[:frame+1], y_array[:frame+1])
    
    # Update robot body
    tr = Affine2D().rotate(curr_theta).translate(curr_x, curr_y) + ax.transData
    body.set_transform(tr)
    
    # Update wheels
    wheel_l.center = (curr_x - (L/2)*np.sin(curr_theta), 
                      curr_y + (L/2)*np.cos(curr_theta))
    wheel_r.center = (curr_x + (L/2)*np.sin(curr_theta), 
                      curr_y - (L/2)*np.cos(curr_theta))
    
    # Heading arrow
    arrow_len = body_length * 0.8
    hx = curr_x + arrow_len * np.cos(curr_theta)
    hy = curr_y + arrow_len * np.sin(curr_theta)
    heading_line.set_data([curr_x, hx], [curr_y, hy])
    
    # Info text
    info_text.set_text(
        f"Time: {time_array[frame]:.2f} s\n"
        f"Position: ({curr_x:.2f}, {curr_y:.2f}) m\n"
        f"Velocity: {curr_v:.2f} m/s\n"
        f"ω: {curr_omega:.2f} rad/s\n"
        f"Error: {path_error[frame]:.3f} m\n"
        f"Power: {total_power[frame]:.1f} W"
    )
    
    return body, wheel_l, wheel_r, path_line, heading_line, info_text

# Animate with frame skipping for smoother playback
frame_skip = 3
frames_to_use = range(0, num_steps, frame_skip)

ani = FuncAnimation(fig, update, frames=frames_to_use, init_func=init,
                    blit=True, interval=50, repeat=False)

# Save animation
out_filename = "figure8_dynamic_simulation.mp4"
try:
    writer = FFMpegWriter(fps=25, bitrate=2000)
    ani.save(out_filename, writer=writer)
    print(f"✅ Animation saved as {out_filename}")
except:
    print("⚠️ FFmpeg not available, using pillow...")
    ani.save(out_filename, writer='pillow', fps=25)
    print(f"✅ Animation saved as {out_filename}")

ani.event_source.stop()
del ani
plt.close('all')

# --- Create Comprehensive Analysis Plots ---
fig2, axes = plt.subplots(4, 3, figsize=(18, 16))
fig2.suptitle('Figure-8 Dynamic Simulation - Complete Analysis', fontsize=18, fontweight='bold')

# Plot 1: Trajectory Comparison
axes[0, 0].plot(x_ref, y_ref, 'g--', linewidth=3, label='Reference', alpha=0.7)
axes[0, 0].plot(x_array, y_array, 'b-', linewidth=2, label='Actual')
axes[0, 0].plot(x_array[0], y_array[0], 'go', markersize=12, label='Start', zorder=5)
axes[0, 0].plot(x_array[-1], y_array[-1], 'ro', markersize=12, label='End', zorder=5)
axes[0, 0].set_xlabel('X Position (m)', fontsize=11)
axes[0, 0].set_ylabel('Y Position (m)', fontsize=11)
axes[0, 0].set_title('1. Trajectory Tracking', fontsize=12, fontweight='bold')
axes[0, 0].legend(loc='best', fontsize=9)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].set_aspect('equal', 'box')

# Plot 2: Path Tracking Error
axes[0, 1].plot(time_array, path_error*100, 'darkred', linewidth=2)
axes[0, 1].fill_between(time_array, 0, path_error*100, alpha=0.3, color='red')
axes[0, 1].set_xlabel('Time (s)', fontsize=11)
axes[0, 1].set_ylabel('Position Error (cm)', fontsize=11)
axes[0, 1].set_title('2. Path Tracking Error', fontsize=12, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
mean_error = np.mean(path_error)*100
axes[0, 1].axhline(y=mean_error, color='orange', linestyle='--', 
                   label=f'Mean: {mean_error:.2f} cm', linewidth=2)
axes[0, 1].legend(loc='best', fontsize=9)

# Plot 3: Heading Error
axes[0, 2].plot(time_array, np.rad2deg(heading_error), 'purple', linewidth=2)
axes[0, 2].axhline(y=0, color='g', linestyle='--', linewidth=2, alpha=0.5)
axes[0, 2].set_xlabel('Time (s)', fontsize=11)
axes[0, 2].set_ylabel('Heading Error (degrees)', fontsize=11)
axes[0, 2].set_title('3. Heading Error', fontsize=12, fontweight='bold')
axes[0, 2].grid(True, alpha=0.3)

# Plot 4: Robot Velocity
axes[1, 0].plot(time_array, v_array, 'b-', linewidth=2.5, label='Actual velocity')
axes[1, 0].plot(t_ref, v_ref, 'g--', linewidth=2, label='Reference velocity', alpha=0.7)
axes[1, 0].set_xlabel('Time (s)', fontsize=11)
axes[1, 0].set_ylabel('Linear Velocity (m/s)', fontsize=11)
axes[1, 0].set_title('4. Robot Body Velocity', fontsize=12, fontweight='bold')
axes[1, 0].legend(loc='best', fontsize=9)
axes[1, 0].grid(True, alpha=0.3)

# Plot 5: Angular Velocity
axes[1, 1].plot(time_array, omega_array, 'darkblue', linewidth=2.5)
axes[1, 1].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[1, 1].set_xlabel('Time (s)', fontsize=11)
axes[1, 1].set_ylabel('Angular Velocity ω (rad/s)', fontsize=11)
axes[1, 1].set_title('5. Robot Angular Velocity', fontsize=12, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)

# Plot 6: Acceleration
axes[1, 2].plot(time_array, a_array, 'red', linewidth=2)
axes[1, 2].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[1, 2].set_xlabel('Time (s)', fontsize=11)
axes[1, 2].set_ylabel('Linear Acceleration (m/s²)', fontsize=11)
axes[1, 2].set_title('6. Linear Acceleration', fontsize=12, fontweight='bold')
axes[1, 2].grid(True, alpha=0.3)

# Plot 7: Driving Force
axes[2, 0].plot(time_array, force_array, 'darkred', linewidth=2)
axes[2, 0].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[2, 0].set_xlabel('Time (s)', fontsize=11)
axes[2, 0].set_ylabel('Net Force (N)', fontsize=11)
axes[2, 0].set_title('7. Net Driving Force', fontsize=12, fontweight='bold')
axes[2, 0].grid(True, alpha=0.3)

# Plot 8: Wheel Torques
axes[2, 1].plot(time_array, torque_left, 'c-', linewidth=2.5, label='Left wheel')
axes[2, 1].plot(time_array, torque_right, 'orange', linewidth=2.5, label='Right wheel')
axes[2, 1].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[2, 1].set_xlabel('Time (s)', fontsize=11)
axes[2, 1].set_ylabel('Torque (N·m)', fontsize=11)
axes[2, 1].set_title('8. Wheel Torques (Motor Torque)', fontsize=12, fontweight='bold')
axes[2, 1].legend(loc='best', fontsize=9)
axes[2, 1].grid(True, alpha=0.3)

# Plot 9: Wheel Linear Velocities
axes[2, 2].plot(time_array, v_left_array, 'c-', linewidth=2.5, label='Left wheel vₗ')
axes[2, 2].plot(time_array, v_right_array, 'orange', linewidth=2.5, label='Right wheel vᵣ')
axes[2, 2].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[2, 2].set_xlabel('Time (s)', fontsize=11)
axes[2, 2].set_ylabel('Velocity (m/s)', fontsize=11)
axes[2, 2].set_title('9. Wheel Linear Velocities', fontsize=12, fontweight='bold')
axes[2, 2].legend(loc='best', fontsize=9)
axes[2, 2].grid(True, alpha=0.3)

# Plot 10: Wheel Angular Velocities
axes[3, 0].plot(time_array, wheel_omega_left, 'c-', linewidth=2.5, label='Left ωₗ')
axes[3, 0].plot(time_array, wheel_omega_right, 'orange', linewidth=2.5, label='Right ωᵣ')
axes[3, 0].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[3, 0].set_xlabel('Time (s)', fontsize=11)
axes[3, 0].set_ylabel('Angular Velocity (rad/s)', fontsize=11)
axes[3, 0].set_title('10. Wheel Angular Velocities', fontsize=12, fontweight='bold')
axes[3, 0].legend(loc='best', fontsize=9)
axes[3, 0].grid(True, alpha=0.3)

# Plot 11: Path Curvature
axes[3, 1].plot(time_array, curvature_array, 'darkgreen', linewidth=2)
axes[3, 1].axhline(y=0, color='k', linestyle='--', alpha=0.3)
axes[3, 1].set_xlabel('Time (s)', fontsize=11)
axes[3, 1].set_ylabel('Curvature κ (1/m)', fontsize=11)
axes[3, 1].set_title('11. Path Curvature (κ = ω/v)', fontsize=12, fontweight='bold')
axes[3, 1].grid(True, alpha=0.3)

# Plot 12: Power Consumption
axes[3, 2].plot(time_array, total_power, 'darkviolet', linewidth=2.5, label='Total power')
axes[3, 2].fill_between(time_array, 0, total_power, alpha=0.3, color='violet')
axes[3, 2].set_xlabel('Time (s)', fontsize=11)
axes[3, 2].set_ylabel('Power (W)', fontsize=11)
axes[3, 2].set_title('12. Total Power Consumption', fontsize=12, fontweight='bold')
axes[3, 2].legend(loc='best', fontsize=9)
axes[3, 2].grid(True, alpha=0.3)
total_energy = np.trapz(total_power, time_array)
axes[3, 2].text(0.95, 0.95, f'Total Energy: {total_energy:.1f} J', 
                transform=axes[3, 2].transAxes, fontsize=9,
                verticalalignment='top', horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plot_filename = "figure8_complete_analysis.png"
plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
print(f"✅ Saved analysis plots -> {plot_filename}")
plt.show()

# Print performance metrics
print("\n" + "="*60)
print("PERFORMANCE METRICS")
print("="*60)
print(f"Mean tracking error: {np.mean(path_error)*100:.2f} cm")
print(f"Max tracking error: {np.max(path_error)*100:.2f} cm")
print(f"RMS tracking error: {np.sqrt(np.mean(path_error**2))*100:.2f} cm")
print(f"Mean heading error: {np.mean(np.abs(heading_error)):.3f} rad ({np.rad2deg(np.mean(np.abs(heading_error))):.2f}°)")
print(f"Total energy consumed: {total_energy:.2f} J")
print(f"Average power: {np.mean(total_power):.2f} W")
print(f"Peak power: {np.max(total_power):.2f} W")
print(f"Max velocity: {np.max(v_array):.2f} m/s")
print(f"Max angular velocity: {np.max(np.abs(omega_array)):.2f} rad/s")
print(f"Max wheel torque: {max(np.max(np.abs(torque_left)), np.max(np.abs(torque_right))):.2f} N·m")
print("="*60)

# Display video
Video(out_filename, embed=True)

Simulating 30s motion with 1500 steps...
✅ Simulation complete!


<IPython.core.display.Javascript object>

✅ Animation saved as figure8_dynamic_simulation.mp4


<IPython.core.display.Javascript object>

C:\Users\Abirami\AppData\Local\Temp\ipykernel_42412\2243307025.py:388: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  total_energy = np.trapz(total_power, time_array)


✅ Saved analysis plots -> figure8_complete_analysis.png

PERFORMANCE METRICS
Mean tracking error: 52.76 cm
Max tracking error: 94.35 cm
RMS tracking error: 57.12 cm
Mean heading error: 0.045 rad (2.56°)
Total energy consumed: 370.25 J
Average power: 12.54 W
Peak power: 592.18 W
Max velocity: 1.43 m/s
Max angular velocity: 1.20 rad/s
Max wheel torque: 33.32 N·m
